In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 26.3 MB/s eta 0:00:00


In [2]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 26.6 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

In [4]:
data=load_dataset('allenai/scitldr')

Generating train split:   0%|          | 0/1992 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/618 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/619 [00:00<?, ? examples/s]

Dataset scitldr downloaded and prepared to /root/.cache/huggingface/datasets/allenai___scitldr/Abstract/0.0.0/79e0fa75961392034484808cfcc8f37deb15ceda153b798c92d9f621d1042fef. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
train=data['train']['source']
test=data['test']['source']

In [7]:
type(train)

list

In [6]:
y=''
for i in range(len(train)):
  d=train[i]
  x=''
  for j in range(len(d)):
    x=x+d[j]
  y=y+x
  
y=y.split(" ")

source,target=[],[]
for i in range(len(y)):
  j=i
  if((i+7>=len(y)) or (j+7>=len(y))):
    break
  if(y[i]!=" "):
    source.append(y[i:i+7])
  else:
    i=i+1
    source.append(y[i:i+7])
  if(y[j+7]!=" "):
    target.append(y[j+7])
  else:
    j=j+1
    target.append(y[j+1])

In [9]:
source[0:3]

[['Due', 'to', 'the', 'success', 'of', 'deep', 'learning'],
 ['to', 'the', 'success', 'of', 'deep', 'learning', 'to'],
 ['the', 'success', 'of', 'deep', 'learning', 'to', 'solving']]

In [10]:
len(source)

304322

In [7]:
import pandas as pd

In [8]:
d1=pd.DataFrame()
d1['text']=source[0:30000]
d1['target']=target[0:30000]

In [13]:
d1.head()

,text,target
0,"[Due, to, the, success, of, deep, learning]",to
1,"[to, the, success, of, deep, learning, to]",solving
2,"[the, success, of, deep, learning, to, solving]",a
3,"[success, of, deep, learning, to, solving, a]",variety
4,"[of, deep, learning, to, solving, a, variety]",of


In [9]:
from transformers import GPTNeoForCausalLM,GPT2Tokenizer,DataCollatorForLanguageModeling,Trainer,TrainingArguments

In [10]:
model=GPTNeoForCausalLM.from_pretrained('EleutherAI/gpt-neo-125M')
tokenizer=GPT2Tokenizer.from_pretrained('EleutherAI/gpt-neo-125M')

In [29]:
tokenizer.pad_token = tokenizer.eos_token
input_ids,attention_mask,labels=[],[],[]
for i in range(d1.shape[0]):
  t=tokenizer(d1['text'][i])
  attention_mask.append(t['attention_mask'])
  input_ids.append(t['input_ids'])
  l=tokenizer(d1['target'][i])['input_ids']
  labels.append(l)

In [30]:
d1['input_ids']=input_ids
d1['attention_mask']=attention_mask
d1['target_labels']=labels

In [31]:
d1.head()

,text,target,input_ids,attention_mask,target_labels
0,"[Due, to, the, success, of, deep, learning]",to,"[[22229], [1462], [1169], [13138], [1659], [22...","[[1], [1], [1], [1], [1], [1], [1]]",[1462]
1,"[to, the, success, of, deep, learning, to]",solving,"[[1462], [1169], [13138], [1659], [22089], [40...","[[1], [1], [1], [1], [1], [1], [1]]","[82, 10890]"
2,"[the, success, of, deep, learning, to, solving]",a,"[[1169], [13138], [1659], [22089], [40684], [1...","[[1], [1], [1], [1], [1], [1], [1, 1]]",[64]
3,"[success, of, deep, learning, to, solving, a]",variety,"[[13138], [1659], [22089], [40684], [1462], [8...","[[1], [1], [1], [1], [1], [1, 1], [1]]","[7785, 1905]"
4,"[of, deep, learning, to, solving, a, variety]",of,"[[1659], [22089], [40684], [1462], [82, 10890]...","[[1], [1], [1], [1], [1, 1], [1], [1, 1]]",[1659]


In [32]:
train=d1[['input_ids','attention_mask','target_labels']]
train.columns=['input_ids','attention_mask','labels']

In [33]:
train.head()

,input_ids,attention_mask,labels
0,"[[22229], [1462], [1169], [13138], [1659], [22...","[[1], [1], [1], [1], [1], [1], [1]]",[1462]
1,"[[1462], [1169], [13138], [1659], [22089], [40...","[[1], [1], [1], [1], [1], [1], [1]]","[82, 10890]"
2,"[[1169], [13138], [1659], [22089], [40684], [1...","[[1], [1], [1], [1], [1], [1], [1, 1]]",[64]
3,"[[13138], [1659], [22089], [40684], [1462], [8...","[[1], [1], [1], [1], [1], [1, 1], [1]]","[7785, 1905]"
4,"[[1659], [22089], [40684], [1462], [82, 10890]...","[[1], [1], [1], [1], [1, 1], [1], [1, 1]]",[1659]
